In [2]:
import torch.utils
import torch.utils.data
import torch.utils.data.dataset
import torchvision
import numpy as np
from torch.utils.data import TensorDataset
import torch
from utils.inputdata import InputData
import random
from utils.runner import Runner
import torch.utils
import torch.utils.data
from models.som import SOM
import models.som
import torch
import matplotlib.pyplot as plt
from utils.inputdata import InputData
import argparse
import wandb
import random
from abc import ABC, abstractmethod
from typing import Any
import importlib
importlib.reload(models.som)
from models.som import SOM
from models.stm import STM
import math
import os

In [7]:

alphas = [5, 2, 1]
betas = [0.005, 0.01, 0.02]
api = wandb.Api()
entity = "replearn"
project = "STM-target-radius-experiment-MNIST"
runs = api.runs(
	path=f"{entity}/{project}",
	filters=None,
	order="-created_at"
)
for run in runs:
# 	run.delete()
	print(run)

	


<Run replearn/STM-target-radius-experiment-MNIST/uvdno50l (running)>
<Run replearn/STM-target-radius-experiment-MNIST/kny9vp3l (running)>
<Run replearn/STM-target-radius-experiment-MNIST/7bap89sq (running)>
<Run replearn/STM-target-radius-experiment-MNIST/c62q40do (running)>
<Run replearn/STM-target-radius-experiment-MNIST/gijrkjrf (finished)>
<Run replearn/STM-target-radius-experiment-MNIST/l15ebc8k (finished)>
<Run replearn/STM-target-radius-experiment-MNIST/eznvyfhu (finished)>
<Run replearn/STM-target-radius-experiment-MNIST/bhoi5mx0 (finished)>
<Run replearn/STM-target-radius-experiment-MNIST/0ws2gdep (finished)>
<Run replearn/STM-target-radius-experiment-MNIST/6nr91s9r (finished)>
<Run replearn/STM-target-radius-experiment-MNIST/pyqo6ama (finished)>
<Run replearn/STM-target-radius-experiment-MNIST/zhw02hko (finished)>
<Run replearn/STM-target-radius-experiment-MNIST/ddi7i1hh (finished)>
<Run replearn/STM-target-radius-experiment-MNIST/cbblum6u (finished)>
<Run replearn/STM-target

In [ ]:
for i, alpha in enumerate(alphas):
	for j, beta in enumerate(betas):
		filters = {

			"state": "finished"
		}
		runs = api.runs(
			path=f"{entity}/{project}",
			filters=None,
			order="-created_at"
		)
		for run in runs:
		 	run.delete()
		